# Advanced Python Constructs

Advanced in the sense that not every language has them, and also that they are more useful in more complicated programs or libraries but not in the sense of being particularly specialized or particularly complicated.

## Iterators, generator expressions and generators

### Iterators

Calling the `__iter__` method on a container to create an iterator object is the most straightforward way to get hold of an iterator. The `iter` function does that for us, saving a few keystrokes.

In [1]:
nums = [1, 2, 3]
iter(nums)

In [2]:
nums.__iter__()

In [3]:
nums.__reversed__()

In [4]:
it = iter(nums)

In [6]:
next(it)

1

In [7]:
next(it)

2

In [8]:
next(it)

3

In [9]:
next(it)

StopIteration: 

When used in a loop, `StopIteration` is swallowed and causes the loop to finish. But with explicit invocation, we can see that once the iterator is exhausted, accessing it raises an exception.  

Using the `for..in` loop also uses the `__iter__` method. This allows us to transparently start the iteration over a sequence. But if we already have the iterator, we want to be able to use it in an for loop in the same way. In order to achieve this, iterators in addition to `next` are also required to have a method called `__iter__` which returns the iterator (`self`).

Support for iteration is pervasive in Python: all sequences and unordered containers in the standard library allow this. The concept is also stretched to other things e.g. file objects support iteration over lines.

In [10]:
f = open('README.md')
f is f.__iter__()

True

In [11]:
f.close()

The `file` is an iterator itself and it's `__iter__` method doesn't create a separate object: only a single thread of sequential access is allowed.

## Generator expressions

A second way in which iterator objects are created is through generator expressions, the basis for list comprehensions. To increase clarity, a generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a list.

In [12]:
(i for i in nums)

<generator object <genexpr> at 0x10e188350>

In [13]:
[i for i in nums]

[1, 2, 3]

In [14]:
list(i for i in nums)

[1, 2, 3]

The list comprehension syntax also extends to dictionary and set comprehensions:

In [18]:
# Set comprehension
s = {i for i in range(3)}
print(s, type(s))

{0, 1, 2} <class 'set'>


In [19]:
# dictionary comprehension
d = {i:i**2 for i in range(3)}
print(d, type(d))

{0: 0, 1: 1, 2: 4} <class 'dict'>


### Generators

A third way to create iterator objects is to call a generator function. A **generator** is a function containing the keyword yield. It must be noted that the mere presence of this keyword completely changes the nature of the function: this `yield` statement doesn't have to be invoked, or even reachable, but causes the function to be marked as a generator. 

When a normal function is called, the instructions contained in the body start to be executed. When a generator is called, the execution stops before the first instruction in the body. An invocation of a generator function creates a generator object, adhering to the iterator protocol.

When next is called, the function is executed until the first yield. Each encountered `yield` statement gives a value becomes the return value of the `next`. After executing the `yield` statement, the execution of this function is suspended.

In [47]:
def f():
    yield 1
    yield 2

In [48]:
f()

<generator object f at 0x10e1b9970>

In [49]:
gen = f()

In [50]:
type(gen)

generator

In [51]:
next(gen)

1

In [52]:
next(gen)

2

In [53]:
try:
    next(gen)
except StopIteration:
    print('Warning! StopIteration')
finally:
    gen.close()

Warning! StopIteration


Let's go over the life of the single invocation of the generator function:

In [41]:
def f():
  print("-- start --")
  yield 3
  print("-- middle --")
  yield 4
  print("-- finished --")

In [42]:
gen = f()

In [43]:
next(gen)

-- start --


3

In [44]:
next(gen)

-- middle --


4

In [45]:
next(gen)

-- finished --


StopIteration: 

Contrary to a normal function, where executing `f()` would immediately cause the first `print` to be executed, `gen` assigned without executing any statements in the function body. Only when `gen.next()` is invoked by `next`, the statements up to the first `yield` are executed... 

<div class="alert alert-block alert-warning">
    When no <em>yield</em> is reached, an exception is raised.
</div>

Why are generators useful? As noted in the parts about iterators, a generator function is just a different way to create an iterator object. Everything that can be done with `yield` statements, could also be done with `next` methods. Nevertheless, using a function and having the interpreter perform its magic to create an iterator has advantages. 

### Bidirectional communication

Each `yield` statement causes a value to be passed to the caller. Communication in the reverse direction is also useful. It is a way to send some external state, either a global variable or a shared mutable object.

The first of the new methods is `send(value)`, which is similar to `next()`, but passes `value` into the generator to be used for the value of the `yield` expression. In fact `g.next()` and `g.send(None)` are equivalent.

The second of the new methods is `throw(type, value=None, traceback=None)` which is equivalent to:

```python
raise type, value, traceback
```

at the point of the `yield` statement.

<div class="alert alert-block alert-info">
Unlike raise (which immediately raises an exception from the current execution point), throw() first resumes the generator, and only then raises the exception.</div>

An exception can be intercepted by an `except` or `finally` clause, or otherwise it causes the execution of the generator function to be aborted and propagates in the caller.  

<div class="alert alert-block alert-info">
Generator iterators also have a close() method, which can be used to force a generator that would otherwise be able to provide more values to finish immediately. It allows the generator __del__ method to destroy objects holding the state of generator.</div>

In [54]:
import itertools
def g():
    print('--start--')
    for i in itertools.count():
        print('--yielding %i--' % i)
        try:
            ans = yield i
        except GeneratorExit:
            print('--closing--')
            raise
        except Exception as e:
            print('--yield raised %r--' % e)
        else:
            print('--yield returned %s--' % ans)

In [55]:
it = g()
next(it)

--start--
--yielding 0--


0

In [59]:
it.send(11)

--yield returned 11--
--yielding 1--


1

In [60]:
next(it)

--yield returned None--
--yielding 2--


2

In [61]:
it.send(22)

--yield returned 22--
--yielding 3--


3

In [62]:
it.close()

--closing--
